In [9]:
%matplotlib inline

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import argparse
import datetime
import importlib
import os
import tensorflow as tf
import yaml
import math
from train_utils import *
from tensorflow.python.framework import graph_util
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [40]:
file_address = open('config/scenenet/test/labels.config')
config = yaml.load(file_address)

/nethome/shickson3/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [41]:
module = importlib.import_module('models.' + config['model'])
model_func = getattr(module, config['model'])
dataset_name = config['dataset_name']
helper = get_dataset(config)
#helper.Setup(config)
modality_infos, num_label_classes = extract_modalities(config)
data_list, iterator = helper.get_test_data(config, num_label_classes)
resnet_name = 'resnet_v2_50'

with tf.variable_scope(resnet_name):
    model = model_func(modality_infos=modality_infos, training=False)
    images_pl, depths_pl, normals_pl, labels_pl, update_ops = setup_model_flops(model, config, train=False)

config1 = tf.ConfigProto()
#config1.gpu_options.allow_growth = True
sess = tf.Session(config=config1)
sess.run(tf.global_variables_initializer())
import_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
print 'total_variables_loaded:', len(import_variables)
#output_graph_def = graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['resnet_v2_50/normals/conv5/batchnorm_1/add_1','resnet_v2_50/Softmax'])
#
#with tf.gfile.GFile('/dev/shm/graph.pb', "wb") as f:
#    f.write(output_graph_def.SerializeToString())

Network_Construction
total_variables_loaded: 383


In [42]:
#def load_pb(pb):
#    with tf.gfile.GFile(pb, "rb") as f:
#        graph_def = tf.GraphDef()
#        graph_def.ParseFromString(f.read())
#    with tf.Graph().as_default() as graph:
#        tf.import_graph_def(graph_def, name='')
#        return graph

#g2 = load_pb('/dev/shm/graph.pb')
#with g2.as_default():
#    run_metadata = tf.RunMetadata()
#    flops = tf.profiler.profile(g2, options=tf.profiler.ProfileOptionBuilder.float_operation())
#print('FLOP = ', flops.total_float_ops)

run_metadata = tf.RunMetadata()
flops = tf.profiler.profile(tf.get_default_graph(), options=tf.profiler.ProfileOptionBuilder.float_operation())
print('FLOP = ', flops.total_float_ops)

('FLOP = ', 40830254434L)


In [43]:
tf.reset_default_graph()

In [44]:
#for n in tf.get_default_graph().as_graph_def().node:
#    print n.name